In [1]:
import cv2
import numpy as np
from keras.models import load_model
from keras.models import model_from_json
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
import keras
print(keras.__version__)

2.3.1


In [3]:
model = load_model('mnist_data.h5')

In [6]:
def keras_predict(model, image):
    processed = keras_process_image(image)
    print("processed: " + str(processed.shape))
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class
    
def keras_process_image(img):
    image_x = 28
    image_y = 28
    print("img2: ",img.shape)
    img = cv2.resize(img, (28,28), interpolation=cv2.INTER_AREA)
    print('img.shape: ', img.shape)
    cv2.imshow("resize",img)
    img = np.array(img, dtype=np.float32)
    print('img.shape2: ', img.shape)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

In [8]:
camera = cv2.VideoCapture(0)
blueLower = np.array([100, 60, 60])
blueUpper = np.array([140, 255, 255])
# print('blueUpper:',blueUpper)
blackboard = np.zeros((780, 740, 3), dtype=np.uint8)
# print(blackboard.shape)
kernel = np.ones((5, 5), np.uint8)
digit = np.zeros((200, 200, 3), dtype=np.uint8)
pts = deque(maxlen=512)
pred_class = 0
# cv2.imshow('video',img)
while True:
    check,frame = camera.read()
    frame = cv2.flip(frame, 1)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    blueMask = cv2.inRange(hsv, blueLower, blueUpper)
    blueMask = cv2.erode(blueMask, kernel, iterations=2)
    blueMask = cv2.morphologyEx(blueMask, cv2.MORPH_OPEN, kernel)
    blueMask = cv2.dilate(blueMask, kernel, iterations=1)
    
#     image,cnts,hierarchy = cv2.findContours(blueMask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts, heir = cv2.findContours(blueMask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    center = None
    if len(cnts)>0:
        cnt = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
        if cv2.contourArea(cnt) > 200:
            ((x, y), radius) = cv2.minEnclosingCircle(cnt)
            cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
            M = cv2.moments(cnt)
            center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
            pts.appendleft(center)
            for i in range(1, len(pts)):
                if pts[i - 1] is None or pts[i] is None:
                    continue
                cv2.line(blackboard, pts[i - 1], pts[i], (255, 255, 255), 7)
                cv2.line(frame, pts[i - 1], pts[i], (0, 255,255), 2)
    elif len(cnts) == 0:
        if len(pts) != []:
            blackboard_gray = cv2.cvtColor(blackboard, cv2.COLOR_BGR2GRAY)
            blur1 = cv2.medianBlur(blackboard_gray, 15)
            blur1 = cv2.GaussianBlur(blur1, (5, 5), 0)
            thresh1 = cv2.threshold(blur1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]  
            blackboard_cnts,black_heir = cv2.findContours(thresh1.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2:]
            if len(blackboard_cnts) >0:
                cnt = max(blackboard_cnts, key=cv2.contourArea)
                if cv2.contourArea(cnt) > 200:
                    x, y, w, h = cv2.boundingRect(cnt)
                    digit = blackboard_gray[y:y + h, x:x + w]
                    pred_probab, pred_class = keras_predict(model, digit)
                    print('predict_class:',pred_class, "predict_proba",pred_probab)
        pts = deque(maxlen=512)        
        blackboard = np.zeros((780,740, 3), dtype=np.uint8)     
    else:
        pts = deque(maxlen=512)
    cv2.putText(frame, str(pred_class), (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.imshow("video",frame)
#     cv2.imshow('black_board',digit)
    k = cv2.waitKey(10)
    if k == 27 :
        break
camera.release()
cv2.destroyAllWindows()

    


img2:  (145, 52)
img.shape:  (28, 28)
img.shape2:  (28, 28)
processed: (1, 28, 28, 1)
predict_class: 1 predict_proba 1.0
img2:  (134, 89)
img.shape:  (28, 28)
img.shape2:  (28, 28)
processed: (1, 28, 28, 1)
predict_class: 2 predict_proba 1.0
img2:  (146, 89)
img.shape:  (28, 28)
img.shape2:  (28, 28)
processed: (1, 28, 28, 1)
predict_class: 3 predict_proba 1.0
img2:  (200, 140)
img.shape:  (28, 28)
img.shape2:  (28, 28)
processed: (1, 28, 28, 1)
predict_class: 4 predict_proba 1.0
img2:  (200, 102)
img.shape:  (28, 28)
img.shape2:  (28, 28)
processed: (1, 28, 28, 1)
predict_class: 5 predict_proba 1.0
img2:  (198, 69)
img.shape:  (28, 28)
img.shape2:  (28, 28)
processed: (1, 28, 28, 1)
predict_class: 6 predict_proba 1.0
img2:  (161, 79)
img.shape:  (28, 28)
img.shape2:  (28, 28)
processed: (1, 28, 28, 1)
predict_class: 7 predict_proba 1.0
img2:  (186, 69)
img.shape:  (28, 28)
img.shape2:  (28, 28)
processed: (1, 28, 28, 1)
predict_class: 8 predict_proba 1.0
img2:  (129, 108)
img.shape:  